In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import random
from scipy import linalg

In [2]:
def decision(df,Dlist,E,K,Llist,H,iterations,lambda1,lambda2,rho):
    W_list=[]
    W=np.random.rand(K-1,H+1)#different from paper
    W_temp=np.random.rand(K-1,H+1)#different from paper
    W_tempnorm=W_temp/(np.linalg.norm(W_temp,'fro'))
    A=np.random.rand(K-1,H+1)#different from paper
    B=np.random.rand(K-1,H+1)#different from paper
    C=np.random.rand(K-1,H+1)#different from paper
    U=np.random.rand(K-1,H+1)
    V=np.random.rand(K-1,H+1)
#     lambda1=.01#hyperparameter
#     lambda2=.001#hyperparameter
#     rho=.0001#hyperparameter
    ind=df.index.values
    flag=True
    for item in range(0,iterations):
        W=calculateW(df,Llist,Dlist,A,U,B,V,ind,rho)
        W=W/(np.linalg.norm(W,'fro'))
#         print(W)
#         print('WWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW')
#         print(U)
# #         print('UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU')
        A=calculateA(C,A,W,U,lambda2,E,rho)
        WU=W+U
#         print(WU)
        B=calculateB(W,V,WU,lambda1,rho)
#         diff=W-W_temp
        print(item,"iteration :",np.linalg.norm((W-W_temp),2))
        W_list.append(np.linalg.norm((W-W_temp),2))
        W_temp=W
        print("***************************************************************")
    print("Done")
    return W,W_list

In [3]:
def calculateW(df,Llist,D_list,A,U,B,V,index,rho):
    for k in range(0,df.shape[0]):
        ts=df.iloc[k]
        Ln=Llist[k]
        Ln=Ln.T
        Dq=D_list[k]
#         Dq=np.nan_to_num(Dq)
        first=np.dot(Ln,Dq)+(0.5*rho*A)-(0.5*rho*U)+(0.5*rho*B)-(0.5*rho*V)
#         print(first)
#         print('#############################')
#         first=np.nan_to_num(first,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
        second_one=np.dot(Dq.T,Dq)#+2*rho*(np.identity(Dq.shape[1])))
#         second_one=np.nan_to_num(second_one,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
        second_second=2*rho*(np.identity(second_one.shape[0]))
#         second_second=np.nan_to_num(second_second,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
        second_temp=second_one+second_second
#         second_temp=np.nan_to_num(second_temp,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
        det = np.linalg.det(second_temp)
        if det!=0:
            second=linalg.inv(second_temp)
        else:
            print('here')
            second=second_temp
#         second=np.nan_to_num(second,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
        w=np.dot(first,second_temp)
#         w=np.nan_to_num(w,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
    return w

In [4]:
def calculateA(C,A,W,U,lambda2,E,rho):
    j=0
    i=0
    while j<W.shape[1]:
        C=W[:,i]+U[:,i]
        temp1=rho*np.linalg.norm(C,2)
        temp2=2*lambda2*E[i]
        temp=temp2/temp1
        temp=1-temp
        if temp<=0:
            A[:,i]=0
        else:
            A[:,i]=temp
#             print('greater than')
#         print('temp',temp)
#         A[:,i]=temp
#         if A[:,i].all()<=0:
#             A[:,i]=0
#         A[:,i]=np.nan_to_num(A[:,i],nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
        i=i+1
        j=j+1
    return A

In [5]:
def calculateB(W,V,WU,lambda1,rho):
    b=np.empty([W.shape[0],W.shape[1]])
    for i in range(0,W.shape[0]):
        for j in range(0,W.shape[1]):
            if WU[i][j]>=((1-lambda1)/rho):
                b[i][j]=(W[i][j]+V[i][j])-((1-lambda1)/rho)
            elif WU[i][j]<=(-1)*((1-lambda1)/rho):
                b[i][j]= (W[i][j]+V[i][j])+((1-lambda1)/rho)
            else:
                b[i][j]=0
#     b=np.nan_to_num(b,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
    return b